# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [4]:
song_files = get_files("data/song_data")
song_filepath = song_files[1]
song_data_df = pd.read_json(song_filepath, lines=True)
song_data = song_data_df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data.head()
song_data.dtypes

,song_id,title,artist_id,year,duration
0,SOGDBUF12A8C140FAA,Intro,AR558FS1187FB45658,2003,75.67628


song_id       object
title         object
artist_id     object
year           int64
duration     float64
dtype: object

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [5]:
for i, row in song_data.iterrows():
    cur.execute(song_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [6]:
artist_data_df = pd.read_json(song_filepath, lines=True)
artist_data = artist_data_df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artist_data.head()


for i, row in artist_data.iterrows():
    cur.execute(artist_table_insert, list(row))
    conn.commit()

,artist_id,artist_name,artist_location,artist_latitude,artist_longitude
0,AR558FS1187FB45658,40 Grit,,NaN,NaN


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [7]:
log_files = get_files("data/log_data")
log_filepath = log_files[1]
logfiledf = pd.read_json(log_filepath, lines=True)
logfiledf.head()

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
1,Jimi Hendrix,Logged In,Mohammad,M,1,Rodriguez,239.82975,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Woodstock Inprovisation,200,1543280209796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
2,Building 429,Logged In,Mohammad,M,2,Rodriguez,300.61669,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Majesty (LP Version),200,1543280448796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88
3,The B-52's,Logged In,Gianna,F,0,Jones,321.54077,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Love Shack,200,1543282396796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38
4,Die Mooskirchner,Logged In,Gianna,F,1,Jones,169.29914,free,"New York-Newark-Jersey City, NY-NJ-PA",PUT,NextSong,1.540870e+12,107,Frisch und g'sund,200,1543282717796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",38


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [8]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [9]:
# logfiledf.count()
# logfiledf['page'].nunique()
logfiledf_filtered=logfiledf[logfiledf['page']=='NextSong']
logfiledf_filtered['datetime']=logfiledf_filtered['ts'].apply(lambda x: pd.to_datetime(x))

logfiledf_filtered['time_data']=logfiledf_filtered['ts'].apply(lambda x: [x
                                                               ,pd.to_datetime(x).hour
                                                               ,pd.to_datetime(x).day
                                                               ,pd.to_datetime(x).week
                                                               ,pd.to_datetime(x).month
                                                               ,pd.to_datetime(x).year
                                                               ,pd.to_datetime(x).weekday()])

logfiledf_filtered.tail()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId,datetime,time_data
298,The Radio Dept.,Logged In,Tegan,F,60,Levine,163.36934,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Where Damage Isn't Already Done,200,1543362012796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,1970-01-01 00:25:43.362012796,"[1543362012796, 0, 1, 1, 1, 1970, 3]"
299,OlÃÂ© OlÃÂ©,Logged In,Tegan,F,61,Levine,247.87546,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Con sÃÂ³lo una mirada,200,1543362175796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,1970-01-01 00:25:43.362175796,"[1543362175796, 0, 1, 1, 1, 1970, 3]"
300,Lady GaGa,Logged In,Tegan,F,62,Levine,274.18077,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Alejandro,200,1543362422796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,1970-01-01 00:25:43.362422796,"[1543362422796, 0, 1, 1, 1, 1970, 3]"
301,The Temper Trap,Logged In,Tegan,F,63,Levine,275.40853,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Fools,200,1543362696796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,1970-01-01 00:25:43.362696796,"[1543362696796, 0, 1, 1, 1, 1970, 3]"
302,The Avett Brothers,Logged In,Tegan,F,64,Levine,244.29669,paid,"Portland-South Portland, ME",PUT,NextSong,1.540794e+12,992,Go To Sleep,200,1543362971796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",80,1970-01-01 00:25:43.362971796,"[1543362971796, 0, 1, 1, 1, 1970, 3]"


In [10]:
time_df = logfiledf_filtered['time_data']
time_df=pd.DataFrame(time_df)
time_df[['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday']]=pd.DataFrame(time_df.time_data.tolist()
                                                                                       ,index= time_df.index)
del time_df['time_data']
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,1543279932796,0,1,1,1,1970,3
1,1543280209796,0,1,1,1,1970,3
2,1543280448796,0,1,1,1,1970,3
3,1543282396796,0,1,1,1,1970,3
4,1543282717796,0,1,1,1,1970,3


In [11]:
time_data = ()
column_labels = ()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [12]:
time_df1=time_df.head(2)

In [13]:
for i, row in time_df1.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [14]:
logfiledf.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [15]:
user_df = logfiledf[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df2=user_df.tail(2)

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [16]:
for i, row in user_df2.iterrows():
    cur.execute(user_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [17]:
# songplay_id, start_time, user_id, level, song_id, artist_id, session_id, location, user_agent
#  ts, userId, level, results[0], results[1], sessionId, location, and userAgent

logfiledf8=logfiledf.head(8)
logfiledf.head(1)

,artist,auth,firstName,gender,itemInSession,lastName,length,level,location,method,page,registration,sessionId,song,status,ts,userAgent,userId
0,Barry Tuckwell/Academy of St Martin-in-the-Fie...,Logged In,Mohammad,M,0,Rodriguez,277.15873,paid,"Sacramento--Roseville--Arden-Arcade, CA",PUT,NextSong,1.540512e+12,961,Horn Concerto No. 4 in E flat K495: II. Romanc...,200,1543279932796,"""Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4...",88


In [20]:
for index, row in logfiledf8.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (i, row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    print(songplay_data)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

(7, 1543279932796, '88', 'paid', None, None, 961, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(7, 1543280209796, '88', 'paid', None, None, 961, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(7, 1543280448796, '88', 'paid', None, None, 961, 'Sacramento--Roseville--Arden-Arcade, CA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"')
(7, 1543282396796, '38', 'free', None, None, 107, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"')
(7, 1543282717796, '38', 'free', None, None, 107, 'New York-Newark-Jersey City, NY-NJ-PA', '"Mozilla/5.0 (Macintosh; Intel Mac

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [21]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.